In [25]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func
from sqlalchemy import and_
from flask import Flask, jsonify
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#create engine
engine = create_engine('sqlite:///C:\\Users\\Madelyn Zelaya\\Desktop\\Module10\\Starter_Code\\Resources\\hawaii.sqlite')

# Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
#print(Base.classes.keys())

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create a session
session = Session(engine)

C:\Users\Madelyn Zelaya\AppData\Local\Temp\ipykernel_33980\3092611168.py:19: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


Precipitation Analysis

In [ ]:
#Find the most recent date in the dataset

latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

# Convert to  datetime 
latest_date = datetime.strptime(latest_date[0], '%Y-%m-%d')

# date one year from the latest date
year_ago = latest_date - timedelta(days=365)

print(f"The most recent date in the dataset is: {latest_date.strftime('%Y-%m-%d')}")


In [ ]:
#the previous 12 months of precipitation data by querying the previous 12 months of data


sel = [Measurement.date, Measurement.prcp]
query_result = session.query(*sel).filter(and_(Measurement.date >= year_ago, Measurement.date <= latest_date)).all()

# Save as a Pandas DataFrame
df = pd.DataFrame(query_result, columns=['date', 'precipitation'])
df.set_index('date', inplace=True)

# Sort  dataframe by date
df = df.sort_values("date")

df.plot(rot=90)
plt.show()

Station Analysis

In [ ]:
# calculate the total number of stations
total_stations = session.query(func.count(Station.station)).scalar()

print(f"The total number of stations in the dataset is: {total_stations}")

In [ ]:
active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).\
    all()

# Print in descending order
for station, count in active_stations:
    print(f"Station: {station}, Observations: {count}")

#  station with the greatest number of observations
most_active_station = active_stations[0][0]

print(f"\nThe station with the greatest number of observations is: {most_active_station}")

In [ ]:
# Query previous 12 months of TOBS data for active station
tobs_data = session.query(Measurement.tobs).\
    filter(and_(Measurement.station == most_active_station, Measurement.date >= year_ago, Measurement.date <= latest_date)).\
    all()

# Save as DataFrame
df = pd.DataFrame(tobs_data, columns=['tobs'])

# histogram with bins=12
df.plot.hist(bins=12)
plt.show()

In [ ]:
session.close()

In [27]:
app = Flask(__name__)

@app.route("/")
def home():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    # Create our session (link) from Python to the DB
    session = Session(engine)
    """Return a list of all precipation queries"""
    # Query
    results = session.query(Measurement.prcp).all()

    session.close()


    # Convert list of tuples into normal list
    precip_data = list(np.ravel(results))

    

#     Return the JSON representation of your dictionary.
    return jsonify(precip_data)

@app.route("/api/v1.0/stations")
def stations():
    active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).\
    all()
    
    return jsonify()

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2023 20:24:32] "GET / HTTP/1.1" 200 -
